In [36]:
import sqlite3
import re

In [37]:
conn_netflix = sqlite3.connect('netflix.sqlite')
cur_netflix = conn_netflix.cursor()
cur_netflix.execute('SELECT title, "cast" FROM netflix_titles')
data = cur_netflix.fetchall()
conn_netflix.close()
new_data = []
uniq_actors = set()
for line in data:
    movie, cast_str = line
    cast = re.split(r' ,|, |,', cast_str)
    uniq_actors = uniq_actors.union(set(cast))
    new_data.append((movie,cast))

In [38]:
conn = sqlite3.connect('task1.sqlite')
cur = conn.cursor()

In [39]:
with conn:
    cur.execute("""
        CREATE TABLE actors (
            id INT NOT NULL PRIMARY KEY,
            name TEXT
        );
    """)

    cur.execute("""
        CREATE TABLE movies (
            id INT NOT NULL PRIMARY KEY,
            name TEXT
        );
    """)

    cur.execute("""
        CREATE TABLE actors_movies (
            user_id INT,
            language_id INT,
            PRIMARY KEY(user_id, language_id),
            FOREIGN KEY(user_id) REFERENCES user(id),
            FOREIGN KEY(language_id) REFERENCES language(id)
        );
    """)


OperationalError: table actors already exists

In [ ]:
with conn:
    for actor in uniq_actors:
        cur.execute('INSERT INTO actors VALUES (NULL, ?)', (actor,))
    
    for line in new_data:
        movie, actors = line
        cur.execute('INSERT INTO movies VALUES (NULL, ?)')

    for i in new_data:
        movie, actors = i
        cur.execute('SELECT id FROM movies WHERE title = ?', (movie,))
        movie_id = cur.fetchone()
        for actor in actors:
            cur.execute('SELECT id FROM actors WHERE name = ?', (actor,))
            actor_id = cur.fetchone()
            cur.execute('INSERT INTO actors_movies (actor_id, movie_id VALUES (?, ?)', (actor_id[0], movie_id[0]))
conn.close()